In [1]:
import tensorflow as tf
import os

from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.utils import build_tensor_info

In [2]:
tf.reset_default_graph()
sess = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))

model_path = './output/model'
ckpt_path = tf.train.latest_checkpoint(model_path)
meta_graph_path = ckpt_path + '.meta'
print(ckpt_path)

saver = tf.train.import_meta_graph(meta_graph_path)
saver.restore(sess, ckpt_path)

graph = tf.get_default_graph()

./output/model/model.ckpt-14000
INFO:tensorflow:Restoring parameters from ./output/model/model.ckpt-14000


In [3]:
x_ph = graph.get_tensor_by_name('test_inputs/x_ph:0')
y_ph = graph.get_tensor_by_name('test_inputs/y_ph:0')

preds = graph.get_tensor_by_name('test_eval/preds:0')
acc = graph.get_tensor_by_name('test_eval/acc:0')
acc_ = graph.get_tensor_by_name('test_eval/acc_:0')

x_ph, y_ph, preds, acc, acc_

(<tf.Tensor 'test_inputs/x_ph:0' shape=(?, 64, 128, 3) dtype=float32>,
 <tf.Tensor 'test_inputs/y_ph:0' shape=(?, 4) dtype=int64>,
 <tf.Tensor 'test_eval/preds:0' shape=(?, 4) dtype=int64>,
 <tf.Tensor 'test_eval/acc:0' shape=() dtype=float32>,
 <tf.Tensor 'test_eval/acc_:0' shape=() dtype=float32>)

In [4]:
export_dir = './output/model_serving/1'
if os.path.exists(export_dir):
    tf.gfile.DeleteRecursively(export_dir)

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

x_ph_info = build_tensor_info(x_ph)
y_ph_info = build_tensor_info(y_ph)
preds_info = build_tensor_info(preds)
acc_info = build_tensor_info(acc)
acc_info_ = build_tensor_info(acc_)

signature_def = signature_def_utils.build_signature_def(
    inputs={'x_ph': x_ph_info, 
           'y_ph': y_ph_info
           },
    outputs={'preds': preds_info, 
            'acc': acc_info,
            'acc_': acc_info_
            },
    method_name=signature_constants.PREDICT_METHOD_NAME
)

builder.add_meta_graph_and_variables(sess, [tag_constants.SERVING],
                                     signature_def_map={
                                         signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
                                     })
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: b'./output/model_serving/1/saved_model.pb'


b'./output/model_serving/1/saved_model.pb'